In [1]:
# Data Manipulation
import pandas as pd # for data manipulation
import numpy as np # for data manipulation

# Training utils
import utils_xgboost

# Optuna
import optuna

# Tiempo
import datetime as dt
from dateutil.relativedelta import relativedelta, MO

# Modelos
from sklearn.linear_model import LinearRegression # for building a linear regression model
from sklearn.svm import SVR # for building SVR model
from sklearn.preprocessing import MinMaxScaler
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Metricas
from sklearn.metrics import mean_absolute_error #MAE
from sklearn.metrics import mean_absolute_percentage_error #MAPE
from sklearn.metrics import mean_squared_error #MSE, para RMSE: squared = False

# Visualizations
import plotly.graph_objects as go # for data visualization
import plotly.express as px # for data visualization
import matplotlib.pyplot as plt

# Advertencias
import warnings
warnings.filterwarnings("ignore")

C:\Users\Fer\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# Cargamos los datos

## solo los datos del set de entrenamiento

In [2]:
datos = pd.read_csv('datos/230127_train_ESTACIONES.csv', parse_dates = ['FECHAHORA'])

# Determinamos las variables de entrenamiento

* Como no tenemos datos de viento, considero que no vale tanto la pena probar en primera instancia metiendo datos de qué estación es, sino entrenar un modelo por estación. 
* El procedimiento será ajustar los hiperparámetros para una sola estación (la 4) y luego entrenar cada estación de manera independiente. 

#### Nota: Si queremos tener en cuenta la interacción de estaciones, posiblemente habría que repensar totalmente la entrada al sistema

## Variables

Entrada: Valores de los 7 días antes de la predicción

Salida: AQI_MP2_5 del día a predecir

variables = [ANHO, DIA, MES, HORA, MINUTO, MP1, MP2_5, MP10, TEMPERATURA, HUMEDAD, PRESION, TEMPERATURA_PRONOSTICO, HUMEDAD_PRONOSTICO, PRESION_PRONOSTICO, DIA_SEM, TRAFICO, AQI_MP2_5, AQI_MP10]


In [4]:
estacion = 4

variables = ["ANHO", 'DIA', 'MES', 'HORA', 'MINUTO', 'MP1', 'MP2_5', 'MP10', 
             'TEMPERATURA', 'HUMEDAD', 'PRESION', 'TEMPERATURA_PRONOSTICO', 
             'HUMEDAD_PRONOSTICO', 'PRESION_PRONOSTICO', 'DIA_SEM', 'TRAFICO' , 'AQI_MP10', 'AQI_MP2_5']

dependent = ['AQI_MP2_5']

number_of_features = len(variables)

training_days = 7 
forecast_days = 1 
samples_per_day = 288
step = 288/24

# Creamos una variable que nos diga con cuantos meses de entrenamiento queremos contar para el X_train
train_months = relativedelta(months = 12)

input_samples = int(samples_per_day * training_days) # cantidad de muestras en 7 dias
output_samples = int(samples_per_day * forecast_days) # cantidad de muestras en 1 dia
train_test_samples = int(input_samples + output_samples) # cantidad de datos para el train_test

In [5]:
# Primero elegimos la estacion que queremos usar 
# y luego dividimos el dataframe en train y test (12 meses y 3 meses)

df_estacion = datos[datos['ESTACION'] == estacion]

df_estacion = df_estacion.reset_index(drop = True)

last_train_set_date = df_estacion.FECHAHORA.min() + train_months

df_train = df_estacion[df_estacion['FECHAHORA'] <= last_train_set_date ].copy()

df_test = df_estacion[df_estacion['FECHAHORA'] > last_train_set_date].copy()

# Por ultimo, elegimos las variables que van a estar en cada set

df_train = df_train[variables]
df_test = df_test[variables]

df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26208 entries, 105409 to 131616
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ANHO                    26208 non-null  int64  
 1   DIA                     26208 non-null  int64  
 2   MES                     26208 non-null  int64  
 3   HORA                    26208 non-null  int64  
 4   MINUTO                  26208 non-null  int64  
 5   MP1                     26208 non-null  float64
 6   MP2_5                   26208 non-null  float64
 7   MP10                    26208 non-null  float64
 8   TEMPERATURA             26208 non-null  float64
 9   HUMEDAD                 26208 non-null  float64
 10  PRESION                 26208 non-null  float64
 11  TEMPERATURA_PRONOSTICO  26208 non-null  float64
 12  HUMEDAD_PRONOSTICO      26208 non-null  float64
 13  PRESION_PRONOSTICO      26208 non-null  float64
 14  DIA_SEM                 26208 no

# Ventanas de entrenamiento (X_train y y_train)

In [6]:
start = 0
end = len(df_train)
next_ = 0

X_train_batches_df = []
y_train_batches_df = []

while (start + train_test_samples) <= end:
    next_ = start + input_samples
    X_train_batches_df.append(df_train.loc[start:next_ - 1, :]) # 20
    y_train_batches_df.append(df_train.loc[next_: next_+output_samples -1, dependent])
    start = start + step
    
X_train_batches = np.asarray(X_train_batches_df)
X_train_batches = X_train_batches.reshape(-1, input_samples, number_of_features)

y_train_batches = np.asarray(y_train_batches_df)
y_train_batches = y_train_batches.reshape(-1, output_samples, 1)

In [7]:
info = X_train_batches.shape 
ynfo = y_train_batches.shape

print(info)
print('Utiliza ' + str(info[0]) + ' entradas. Cada una contiene ' + str(info[1]) + 
      ' puntos de medicion. Cada punto de medicion tiene ' + str(info[2]) + ' features')

print('\n')

print(ynfo)
print('Predice ' + str(ynfo[0]) + ' salidas. Cada una entrega ' + str(ynfo[1]) + 
      ' puntos de medicion. Cada punto de medicion tiene ' + str(ynfo[2]) + ' feature')

(8593, 2016, 18)
Utiliza 8593 entradas. Cada una contiene 2016 puntos de medicion. Cada punto de medicion tiene 18 features


(8593, 288, 1)
Predice 8593 salidas. Cada una entrega 288 puntos de medicion. Cada punto de medicion tiene 1 feature


# Ventanas de testeo

In [8]:
start = df_test.index[0]
end = len(df_test) + start
next_ = 0

X_test_batches_df = []
y_test_batches_df = []

while (start + train_test_samples) <= end:
    next_ = start + input_samples
    X_test_batches_df.append(df_test.loc[start:next_ - 1, :]) # 20
    y_test_batches_df.append(df_test.loc[next_: next_+output_samples -1, dependent])
    start = start + step
    
X_test_batches = np.asarray(X_test_batches_df)
X_test_batches = X_test_batches.reshape(-1, input_samples, number_of_features)

y_test_batches = np.asarray(y_test_batches_df)
y_test_batches = y_test_batches.reshape(-1, output_samples, 1)

In [9]:
len(X_train_batches[0])

2016

In [10]:
info = X_test_batches.shape 
ynfo = y_test_batches.shape

print(info)
print('Utiliza ' + str(info[0]) + ' entradas. Cada una contiene ' + str(info[1]) + 
      ' puntos de medicion. Cada punto de medicion tiene ' + str(info[2]) + ' features')

print('\n')

print(ynfo)
print('Predice ' + str(ynfo[0]) + ' salidas. Cada una entrega ' + str(ynfo[1]) + 
      ' puntos de medicion. Cada punto de medicion tiene ' + str(ynfo[2]) + ' feature')

(1993, 2016, 18)
Utiliza 1993 entradas. Cada una contiene 2016 puntos de medicion. Cada punto de medicion tiene 18 features


(1993, 288, 1)
Predice 1993 salidas. Cada una entrega 288 puntos de medicion. Cada punto de medicion tiene 1 feature


# Cambiar cada ventana a una instancia

In [11]:
X_train = []

for i in range(0, len(X_train_batches)):
    
    hold = []
    
    for j in range(0, len(X_train_batches[i])):
        
        if j==(len(X_train_batches[i]) - 1):
            hold = np.concatenate((hold, X_train_batches[i][j][:]), axis = None)
            
        else:
            hold = np.concatenate((hold, X_train_batches[i][j][-1]), axis = None)
            
    X_train.append(hold)
        
X_test = []

for i in range(0, len(X_test_batches)):
    
    hold = []
    
    for j in range(0, len(X_test_batches[i])):
        
        if j==(len(X_test_batches[i]) -1):
            #print('hola')
            hold = np.concatenate((hold, X_test_batches[i][j][:]), axis = None)
            
        else:
            hold = np.concatenate((hold, X_test_batches[i][j][-1]), axis = None)
            
    X_test.append(hold)
    
X_train = np.reshape(X_train, (len(X_train), len(X_train[0])))

y_train = np.reshape(y_train_batches, (len(y_train_batches), output_samples))

X_test = np.reshape(X_test, (len(X_test), len(X_test[0])))

y_test = np.reshape(y_test_batches, (len(y_test_batches), output_samples))

In [12]:
print(X_train.shape)
print(y_train.shape)

(8593, 2033)
(8593, 288)


In [13]:
X_train[0][-30:-1]

array([3.3000e+01, 3.3000e+01, 3.3000e+01, 3.3000e+01, 3.3000e+01,
       3.3000e+01, 3.3000e+01, 3.3000e+01, 3.3000e+01, 3.3000e+01,
       3.3000e+01, 3.3000e+01, 2.0190e+03, 7.0000e+00, 4.0000e+00,
       1.2000e+01, 1.5000e+01, 4.2500e+00, 6.3290e+00, 6.7190e+00,
       2.8300e+01, 4.7000e+01, 1.0072e+03, 2.8400e+01, 5.1700e+01,
       1.0059e+03, 6.0000e+00, 1.0000e+00, 1.3000e+01])

# Entrenamiento XGBOOST

In [14]:
params = {'learning_rate' : 0.025,
          'n_estimators' : 250,
          'max_depth': 2,
          'min_child_weight' : 1,
          'gamma': 0.0,
          'subsample': 0.98,
          'colsample_bytree': 0.98,
          'scale_pos_weight': 0.8,
          'seed': 42,
          'verbosity' : 0}


xgb_model = xgb.XGBRegressor(** params)

trained_xgb_model = MultiOutputRegressor(xgb_model).fit(X_train , y_train)

In [15]:
prediction = trained_xgb_model.predict(X_test)
print('prediction', prediction.shape)
print('test', y_test.shape)



prediction (1993, 288)
test (1993, 288)


In [16]:
MSE = np.mean((prediction - y_test)**2)
print('RMSE: ', MSE**0.5)
MAE = np.mean(np.abs(prediction-y_test))
print('MAE: ', MAE)

RMSE:  9.65785112939056
MAE:  6.633150163845001


In [28]:
y_test_prueba = y_test[: -1]
print(len(y_test_prueba))

y_pred_prueba = prediction[: -1]
len(y_pred_prueba)


1992


1992

In [42]:
# y_test_dividida = np.reshape(y_test_prueba, (24, 83 ,288))
# y_pred_dividida = np.reshape(y_pred_prueba, (24, 83, 288))

# y_test_a = y_test_dividida[1][:][:]
# y_pred_a = y_pred_dividida[1][:][:]

# y_test_a.shape

pred_1 = []


for i in range(0, len(y_pred_prueba), 24):
    pred_1.append(y_pred_prueba[i])
    
pred_1 = np.asarray(pred_1)
    
test_1 = []

for i in range(0, len(y_test_prueba), 24):
    test_1.append(y_test_prueba[i])
    
test_1 = np.asarray(test_1)

print(pred_1.shape)
print(test_1.shape)

(83, 288)
(83, 288)


In [43]:
y_pred = np.reshape(pred_1, ( len(pred_1) * len(pred_1[0])))
y_test_1 = np.reshape(test_1, (len(test_1) * len(test_1[0])))

print(y_pred)
print(y_test_1)

fig_val = go.Figure()

fig_val.add_trace(
    go.Scatter( y = list(y_test_1), name = 'Target'))

fig_val.add_trace(
     go.Scatter( y = list(y_pred), name = 'Forecasts'))

fig_val.update_layout( title_text = "Validation - Forecasts vs Targets")

[12.078805 12.121002 12.129887 ... 34.97305  35.027405 35.416515]
[12. 12. 12. ... 12. 12. 12.]


In [44]:
MSE = np.mean((y_pred - y_test_1)**2)
print('RMSE: ', MSE**0.5)
MAE = np.mean(np.abs(y_pred-y_test_1))
print('MAE: ', MAE)

RMSE:  10.250435791778498
MAE:  6.542418526278602
